In [0]:
file_location = "dbfs:/FileStore/shared_uploads/cavrudy@gmail.com/survey_responses-1.csv"
file_location2 = "dbfs:/FileStore/shared_uploads/cavrudy@gmail.com/conversion2dollar.csv"
file_type = "csv"

infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

In [0]:
from pyspark.sql.types import IntegerType
import pyspark.sql.functions as f

#1. Create the DataFrames
responsesDF  = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

ratesDF = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location2)

responsesDF = responsesDF.fillna({'CompensationAmount':0})
responsesDF = responsesDF.withColumn("Compensation", f.regexp_replace(f.col("CompensationAmount"), "[,]", "").cast(IntegerType()))
responsesDF = responsesDF.drop("CompensationAmount")

display(responsesDF.select("Compensation", "CompensationCurrency", "GenderSelect").describe())

summary,Compensation,CompensationCurrency,GenderSelect
count,16703,4530,16620
mean,832237.6677842304,70787.0,null
stddev,1.7657423860503115E7,41313.42079760523,null
min,-99,100000,A different identity
max,1194000000,ZAR,"Non-binary, genderqueer, or gender non-conforming"


In [0]:
#What is the gender split among the participants by Country
responsesDF.groupby('Country', "GenderSelect").count().show()

+--------------------+--------------------+-----+
|             Country|        GenderSelect|count|
+--------------------+--------------------+-----+
|              Sweden|              Female|    8|
|               Japan|                Male|  254|
|              Norway|              Female|    5|
|           Hong Kong|                Male|   52|
|         Philippines|              Female|   22|
|      United Kingdom|              Female|   88|
|        South Africa|A different identity|    2|
|People 's Republi...|A different identity|    3|
|             Germany|              Female|   61|
|              Sweden|                Male|   81|
|            Pakistan|              Female|   28|
|              Poland|                Male|  160|
|           Argentina|                Male|   79|
|People 's Republi...|              Female|   74|
|             Ireland|                Male|   64|
|              Israel|                Male|   93|
|               Italy|              Female|   27|


In [0]:
#3. Join the data with the conversion rates to have all the amount entered in the 'Compensation' field in US Dollar
responseUSDF = responsesDF.join(ratesDF, responsesDF.CompensationCurrency == ratesDF.originCountry)
responseUSDF = responseUSDF.withColumn("CompensationAmountUS",responseUSDF["Compensation"]*responseUSDF["exchangeRate"])

responseUSDF.select("CompensationCurrency","Compensation","CompensationAmountUS").show(20)

+--------------------+------------+--------------------+
|CompensationCurrency|Compensation|CompensationAmountUS|
+--------------------+------------+--------------------+
|                 USD|      250000|            250000.0|
|                 AUD|       80000|  64184.799999999996|
|                 RUB|     1200000|             20882.4|
|                 INR|       95000|              1483.9|
|                 TWD|     1100000|             36634.4|
|                 USD|       20000|             20000.0|
|                 USD|      100000|            100000.0|
|                 CZK|       20000|               916.4|
|                 RUB|      624000|           10858.848|
|                 COP|   156000000|             53352.0|
|                 EUR|      150000|            179373.9|
|                 PLN|      126000|           35419.104|
|                 GBP|      130000|           172144.44|
|                 USD|      133000|            133000.0|
|                 USD|       80

In [0]:
#4. Is there a difference on the average salary among the different gender groups? 
responseUSDF.registerTempTable("responseTbl")

salaryDF = sqlContext.sql("SELECT GenderSelect, ROUND(AVG(CompensationAmountUS), 0) AS AverageSalary , ROUND(PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY CompensationAmountUS),0) AS MedianSalary \
                           FROM responseTbl WHERE GenderSelect IS NOT NULL GROUP BY GenderSelect").display()

# There is a difference among gender groups between average salary, which was higher than the median salary. Males had the highest avgerage salary while a different identity had the highest median salary

/databricks/spark/python/pyspark/sql/dataframe.py:146: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn(


GenderSelect,AverageSalary,MedianSalary
Female,58410.0,47833.0
"Non-binary, genderqueer, or gender non-conforming",62535.0,42000.0
Male,71582.0,51991.0
A different identity,100261.0,58595.0


In [0]:
#5.1 Using the Age and CompensationAmountUS fields, try to determine that type of relationship (if any) exists between Age and CompensationAmountUS
from pyspark.sql.functions import round, mean, desc, asc, floor, avg
import matplotlib.pyplot as plt

responseUSDF.na.drop(subset=["Age"]).groupBy("Age").agg(round(mean("CompensationAmountUS"),0).alias("CompensationAmount")).sort(asc("Age")).display()

#There is a linear correlation between Compensation amount and Age. Compensation tends to increase as age increases

Age,CompensationAmount
0,62580.0
100,null
11,0.0
16,50000.0
18,20500.0
19,14069.0
20,27906.0
21,21809.0
22,24364.0
23,27737.0


In [0]:
#5.2 What is the average salary in US for people in their 20s? And for people in their 50s?

from pyspark.sql import functions as F

age_table = sqlContext.sql("SELECT Age/10 AS Age, CompensationAmountUS FROM responseTbl")
age_table = age_table.withColumn("decade",(floor(age_table["Age"])*10)).groupby('decade').agg(round(F.avg("CompensationAmountUS"),0).alias('AverageCompensation')).sort(asc("decade")).display()

#Average salary for people in their 20s: $45,773
#                                   50s: $105,253


decade,AverageCompensation
null,76099.0
0,62580.0
10,17927.0
20,45773.0
30,76156.0
40,94769.0
50,105253.0
60,110523.0
70,95510.0
90,17937.0


In [0]:
#6. What is the job with the highest compensation?
salaryByJobDF = responseUSDF.na.drop(subset=["CurrentJobTitleSelect"]).groupBy("CurrentJobTitleSelect").\
    agg(round(F.percentile_approx("CompensationAmountUS", 0.5),0).alias("MedianCompensation"),\
        round(avg('CompensationAmountUS'), 0).alias('AverageCompensation')).sort(desc("AverageCompensationCurrentJobTitleSelectAverageCompensation")).display()

#Operations Research Practitioner had both the highest and median compensation 

CurrentJobTitleSelect,MedianCompensation,AverageCompensation
Operations Research Practitioner,98843.0,151626.0
Other,65000.0,90192.0
Software Developer/Software Engineer,52968.0,86688.0
Data Scientist,68850.0,78466.0
Statistician,46000.0,77661.0
Predictive Modeler,65000.0,73884.0
Business Analyst,41595.0,70008.0
Machine Learning Engineer,41765.0,61575.0
Engineer,50000.0,60933.0
Scientist/Researcher,43050.0,59442.0


In [0]:
#7. What is the country with the highest compensation?

responseUSDF.groupBy("Country").\
    agg(round(F.percentile_approx("CompensationAmountUS", 0.5),0).alias("MedianCompensation"), \
        round(avg('CompensationAmountUS'),0).alias('AverageCompensation')).sort(desc("MedianCompensation")).display()

#United States had the highest median compensation, but the People's Republic of China had the highest average compensation


Country,MedianCompensation,AverageCompensation
United States,105000.0,117275.0
Switzerland,104338.0,104291.0
Australia,88254.0,96933.0
Norway,86064.0,88413.0
Denmark,80365.0,73841.0
Israel,74705.0,81121.0
Netherlands,71750.0,77424.0
Germany,71750.0,71866.0
Canada,68367.0,73336.0
United Kingdom,66209.0,90578.0
